<a href="https://colab.research.google.com/github/nrutyadoshi/NLP/blob/Homework/NLP_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import files
files = files.upload()

Saving spam.csv to spam (2).csv


In [0]:
import pandas as pd
import numpy as np

In [0]:
spam = pd.read_csv("spam (2).csv")

In [43]:
spam.shape

(5572, 5)

In [44]:
spam.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [0]:
spam.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], 1,inplace=True)

In [46]:
spam.isnull().sum()

v1    0
v2    0
dtype: int64

In [47]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [0]:
import re

In [0]:
def text_data(str):
  str_new = " ".join(re.findall("[a-zA-Z]+", str))
  return (str_new)

In [0]:
spam["v2"] = spam.apply(lambda row: text_data(row.v2),axis=1)

In [0]:
def tokenize(str_new):
  tokens = []
  tokens = nltk.word_tokenize(str_new)
  return (tokens)

In [0]:
spam["v2"] = spam.apply(lambda row: tokenize(row.v2),axis=1)

In [0]:
from nltk.corpus import stopwords

In [0]:
def remove(tokens):
  removed = []
  stopword = stopwords.words('english') 
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for w in tokens:
    if (w.lower() not in stopword and w.lower() not in punctuations):
      removed.append(w.lower())
  return (removed)

In [0]:
spam["v2"] = spam.apply(lambda row: remove(row.v2),axis=1)

In [0]:
def tagging(removed):
  pos = []
  pos.append(nltk.pos_tag(removed))
  return (pos)

In [0]:
spam["v2"] = spam.apply(lambda row: tagging(row.v2),axis=1)

In [0]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [0]:
def lemmatization(pos):
  lem = WordNetLemmatizer()
  lemma = []
  for word in pos:
    for w in word:
      pos_value = ""
      if (w[1].startswith('J')):
        pos_value = wordnet.ADJ
      elif (w[1].startswith('V')):
        pos_value = wordnet.VERB
      elif (w[1].startswith('N')):
        pos_value = wordnet.NOUN
      elif (w[1].startswith('R')):
        pos_value = wordnet.ADV 
      else:
        continue
      lemma.append(lem.lemmatize(w[0],pos_value))
  return (lemma)

In [0]:
spam["v2"] = spam.apply(lambda row: lemmatization(row.v2),axis=1)

In [61]:
spam.head()

,v1,v2
0,ham,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, wkly, comp, win, fa, cup, final,..."
3,ham,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"[nah, think, go, usf, life]"


In [0]:
def to_string(lemma):
  str1 = ' '.join([elem for elem in lemma]) 
  return (str1)

In [0]:
spam["v2"] = spam.apply(lambda row: to_string(row.v2),axis=1)

In [73]:
spam.head()

,v1,v2
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry wkly comp win fa cup final tkts st ...
3,ham,u dun say early hor u c already say
4,ham,nah think go usf life


In [0]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, Y_train, Y_test = tts(spam['v2'],spam['v1'])

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier 

In [0]:
from sklearn.pipeline import Pipeline

In [0]:
steps = [('text',CountVectorizer()),('ensemble',RandomForestClassifier())]
pipeline = Pipeline(steps)

In [85]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('text',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

In [86]:
pipeline.score(X_test, Y_test)

0.9784637473079684